In [1]:
import tensorflow as tf
import numpy as np
import json
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [2]:
# Load the transformer model
transformer = tf.keras.models.load_model('transformer_model')

# Load the tokenization configuration and vocabulary for English and Spanish
with open('eng_vectorization_config.json', 'r', encoding='utf-8') as f:
    eng_vectorization_config = json.load(f)
    max_len = eng_vectorization_config.get('max_len', 20)  # Default to 20 if 'max_len' is missing

with open('eng_vocab.json', 'r', encoding='utf-8') as f:
    eng_vocab = json.load(f)

with open('spa_vectorization_config.json', 'r', encoding='utf-8') as f:
    spa_vectorization_config = json.load(f)
    spa_max_len = spa_vectorization_config.get('max_len', 20)  # Default to 20 if 'max_len' is missing

with open('spa_vocab.json', 'r', encoding='utf-8') as f:
    spa_vocab = json.load(f)

# Initialize tokenizers
eng_tokenizer = Tokenizer()
eng_tokenizer.word_index = {word: index for index, word in enumerate(eng_vocab)}

spa_tokenizer = Tokenizer()
spa_tokenizer.word_index = {word: index for index, word in enumerate(spa_vocab)}

# Define the function for vectorizing English input sentences
def eng_vectorization(sentences):
    sequences = eng_tokenizer.texts_to_sequences(sentences)
    return pad_sequences(sequences, maxlen=max_len, padding='post')

# Define BeamSearchDecoder class with corrected __init__ method
class BeamSearchDecoder:
    def __init__(self, model, tokenizer, beam_width, max_seq_len):
        self.model = model
        self.tokenizer = tokenizer
        self.beam_width = beam_width
        self.max_seq_len = max_seq_len
        self.start_token = tokenizer.word_index.get('[start]', 0)
        self.end_token = tokenizer.word_index.get('[end]', 0)
        self.vocab_size = len(tokenizer.word_index)

    def decode(self, input_seq):
        # Initialize the beam with the start token
        beam = [(input_seq, [self.start_token], 0)]
        
        # Loop until the beam reaches the maximum sequence length
        for _ in range(self.max_seq_len):
            new_beam = []
            for input_seq, output_seq, score in beam:
                # Expand the current output sequence
                output_seq_padded = pad_sequences([output_seq], maxlen=self.max_seq_len, padding='post')
                predictions = self.model.predict([input_seq, output_seq_padded], verbose=0)
                
                # Get the top-k predictions
                top_k_indices = np.argsort(predictions[0, len(output_seq)-1, :])[-self.beam_width:]
                for index in top_k_indices:
                    new_score = score + np.log(predictions[0, len(output_seq)-1, index])
                    new_seq = output_seq + [index]
                    new_beam.append((input_seq, new_seq, new_score))
            
            # Keep the best beam_width sequences
            beam = sorted(new_beam, key=lambda x: x[2], reverse=True)[:self.beam_width]
            
            # If the end token is reached, break
            if all(seq[-1] == self.end_token for _, seq, _ in beam):
                break

        # Return the best sequence
        return beam[0][1]

def sequence_to_text(sequence, tokenizer):
    index_to_word = {index: word for word, index in tokenizer.word_index.items()}
    return ' '.join([index_to_word.get(index, '') for index in sequence if index != 0 and index != tokenizer.word_index.get('[start]', -1) and index != tokenizer.word_index.get('[end]', -1)])

# Translate function using beam search
def translate_sentence(sentence, beam_width=3, max_seq_len=20):
    input_seq = eng_vectorization([sentence])
    beam_search_decoder = BeamSearchDecoder(transformer, spa_tokenizer, beam_width, max_seq_len)
    decoded_sequence = beam_search_decoder.decode(input_seq)
    return sequence_to_text(decoded_sequence, spa_tokenizer)

In [3]:
# Example usage and printing the translated sentence
sentence = "that cat was my most loved animal"
translated_sentence = translate_sentence(sentence)
print(f"Original sentence: {sentence}")
print(f"Translated sentence: {translated_sentence}")

Original sentence: that cat was my most loved animal
Translated sentence: ese gato era el canadiense
